# 🧠 HyperCat Algebraic Topology Demo
This notebook demonstrates advanced use of the HyperCat category theory framework in algebraic topology.

In [1]:
# Import the core HyperCat components
from hypercat import Category, Object, Morphism, Functor, NaturalTransformation
from hypercat.categories import StandardCategories
from hypercat.higher import TwoCell, TwoCategory

# Additional imports for the topology demo
from typing import Any, Dict, List, Set, Tuple, Optional, Callable, Union, Generic, TypeVar
from dataclasses import dataclass
import itertools
import copy
from collections import defaultdict
from abc import ABC, abstractmethod

T = TypeVar('T')
U = TypeVar('U')

# Topos Theory Elements
class Topos(Category):
    """Represents an elementary topos."""
    
    def __init__(self, name: str):
        super().__init__(name)
        self.terminal_object: Optional[Object] = None
        self.subobject_classifier: Optional[Object] = None
        self.truth_morphism: Optional[Morphism] = None
    
    def set_terminal_object(self, obj: Object) -> 'Topos':
        """Set the terminal object."""
        self.terminal_object = obj
        return self
    
    def set_subobject_classifier(self, omega: Object, true_morph: Morphism) -> 'Topos':
        """Set the subobject classifier Ω and true: 1 -> Ω."""
        self.subobject_classifier = omega
        self.truth_morphism = true_morph
        return self
    
    def has_finite_limits(self) -> bool:
        """Check if the topos has finite limits (simplified check)."""
        return self.terminal_object is not None
    
    def has_exponentials(self) -> bool:
        """Check if the topos has exponentials (simplified check)."""
        # In a real implementation, you'd check for exponential objects
        return True  # Assume true for elementary toposes


# Enriched Categories
class EnrichedCategory:
    """Category enriched over a monoidal category."""
    
    def __init__(self, name: str, enriching_category: Category):
        self.name = name
        self.enriching_category = enriching_category
        self.objects: Set[Object] = set()
        self.hom_objects: Dict[Tuple[Object, Object], Object] = {}
        self.composition_morphisms: Dict[Tuple[Object, Object, Object], Morphism] = {}
        self.unit_morphisms: Dict[Object, Morphism] = {}
    
    def add_object(self, obj: Object) -> 'EnrichedCategory':
        """Add an object to the enriched category."""
        self.objects.add(obj)
        return self
    
    def set_hom_object(self, A: Object, B: Object, hom_obj: Object) -> 'EnrichedCategory':
        """Set the hom-object [A,B] in the enriching category."""
        self.hom_objects[(A, B)] = hom_obj
        return self
    
    def set_composition(self, A: Object, B: Object, C: Object, 
                       comp_morph: Morphism) -> 'EnrichedCategory':
        """Set composition morphism [B,C] ⊗ [A,B] -> [A,C]."""
        self.composition_morphisms[(A, B, C)] = comp_morph
        return self


# Homotopy Type Theory Elements  
class HomotopyType:
    """Represents a homotopy type."""
    
    def __init__(self, name: str, level: int = 0):
        self.name = name
        self.level = level  # h-level: 0=contractible, 1=proposition, 2=set, etc.
        self.paths: Dict[Tuple[Any, Any], 'HomotopyType'] = {}
    
    def add_path_type(self, a: Any, b: Any, path_type: 'HomotopyType') -> 'HomotopyType':
        """Add a path type between two points."""
        self.paths[(a, b)] = path_type
        return self
    
    def is_contractible(self) -> bool:
        """Check if this is a contractible type (h-level 0)."""
        return self.level == 0
    
    def is_proposition(self) -> bool:
        """Check if this is a proposition (h-level 1)."""
        return self.level <= 1


class InfinityCategory:
    """Represents an (∞,1)-category."""
    
    def __init__(self, name: str):
        self.name = name
        self.objects: Set[Object] = set()
        # Use lists instead of sets to store morphisms with dictionaries
        self.morphisms: Dict[int, List] = defaultdict(list)  # n-morphisms by dimension
        self.higher_compositions: Dict = {}
    
    def add_object(self, obj: Object) -> 'InfinityCategory':
        """Add an object (0-morphism)."""
        self.objects.add(obj)
        return self
    
    def add_n_morphism(self, n: int, morph: Any) -> 'InfinityCategory':
        """Add an n-morphism."""
        self.morphisms[n].append(morph)  # Use append instead of add
        return self
    
    def nerve(self) -> 'SimplicalSet':
        """Compute the nerve of the ∞-category."""
        # Simplified implementation
        return SimplicalSet(f"Nerve({self.name})")


class SimplicalSet:
    """Represents a simplicial set."""
    
    def __init__(self, name: str):
        self.name = name
        # Use lists for simplices to avoid hashability issues
        self.simplices: Dict[int, List] = defaultdict(list)  # n-simplices
        self.face_maps: Dict = {}
        self.degeneracy_maps: Dict = {}
    
    def add_simplex(self, n: int, simplex: Any) -> 'SimplicalSet':
        """Add an n-simplex."""
        self.simplices[n].append(simplex)  # Use append instead of add
        return self
    
    def geometric_realization(self) -> 'TopologicalSpace':
        """Compute geometric realization (simplified)."""
        return TopologicalSpace(f"|{self.name}|")


class TopologicalSpace:
    """Represents a topological space."""
    
    def __init__(self, name: str):
        self.name = name
        self.points: Set = set()
        self.open_sets: Set = set()
    
    def fundamental_groupoid(self) -> 'Groupoid':
        """Compute the fundamental groupoid."""
        return Groupoid(f"Π₁({self.name})")


class Groupoid(Category):
    """Represents a groupoid (category where all morphisms are isomorphisms)."""
    
    def __init__(self, name: str):
        super().__init__(name)
    
    def add_morphism(self, morph: Morphism) -> 'Groupoid':
        """Add a morphism and its inverse."""
        super().add_morphism(morph)
        
        # Add inverse if not identity
        if morph.source != morph.target:
            inv_name = f"{morph.name}⁻¹"
            if not any(m.name == inv_name for m in self.morphisms):
                inv_morph = Morphism(inv_name, morph.target, morph.source)
                super().add_morphism(inv_morph)
                
                # Set up inverse composition
                if morph.source in self.identities and morph.target in self.identities:
                    self.set_composition(morph, inv_morph, self.identities[morph.source])
                    self.set_composition(inv_morph, morph, self.identities[morph.target])
        
        return self


# Advanced Category Constructions
class MonoidalCategory(Category):
    """Represents a monoidal category."""
    
    def __init__(self, name: str):
        super().__init__(name)
        self.tensor_product: Optional[Callable[[Object, Object], Object]] = None
        self.unit_object: Optional[Object] = None
        self.associator: Dict = {}
        self.left_unitor: Dict = {}
        self.right_unitor: Dict = {}
    
    def set_tensor_product(self, tensor_func: Callable[[Object, Object], Object]) -> 'MonoidalCategory':
        """Set the tensor product operation."""
        self.tensor_product = tensor_func
        return self
    
    def set_unit_object(self, unit: Object) -> 'MonoidalCategory':
        """Set the unit object for the tensor product."""
        self.unit_object = unit
        return self
    
    def tensor_objects(self, obj1: Object, obj2: Object) -> Optional[Object]:
        """Compute tensor product of objects."""
        if self.tensor_product:
            return self.tensor_product(obj1, obj2)
        return None


class BraidedMonoidalCategory(MonoidalCategory):
    """Represents a braided monoidal category."""
    
    def __init__(self, name: str):
        super().__init__(name)
        self.braiding: Dict[Tuple[Object, Object], Morphism] = {}
    
    def set_braiding(self, obj1: Object, obj2: Object, braiding_morph: Morphism) -> 'BraidedMonoidalCategory':
        """Set the braiding morphism β_{A,B}: A⊗B -> B⊗A."""
        self.braiding[(obj1, obj2)] = braiding_morph
        return self


class SymmetricMonoidalCategory(BraidedMonoidalCategory):
    """Represents a symmetric monoidal category."""
    
    def __init__(self, name: str):
        super().__init__(name)
    
    def is_symmetric(self) -> bool:
        """Check if braiding is symmetric (β_{B,A} ∘ β_{A,B} = id)."""
        # Simplified check
        return True


# Operads and Algebraic Structures
class Operad:
    """Represents an operad."""
    
    def __init__(self, name: str):
        self.name = name
        # Use lists instead of sets for operations
        self.operations: Dict[int, List] = defaultdict(list)  # n-ary operations
        self.composition: Dict = {}
        self.unit: Optional[Any] = None
    
    def add_operation(self, arity: int, operation: Any) -> 'Operad':
        """Add an operation of given arity."""
        self.operations[arity].append(operation)  # Use append instead of add
        return self
    
    def set_unit(self, unit_op: Any) -> 'Operad':
        """Set the unit operation."""
        self.unit = unit_op
        return self


class Algebra:
    """Represents an algebra over an operad."""
    
    def __init__(self, name: str, operad: Operad, underlying_object: Object):
        self.name = name
        self.operad = operad
        self.underlying_object = underlying_object
        self.structure_maps: Dict = {}
    
    def set_structure_map(self, operation: Any, structure_morph: Morphism) -> 'Algebra':
        """Set how an operad operation acts on the algebra."""
        self.structure_maps[operation] = structure_morph
        return self

In [ ]:
def run_algebraic_topology_demo():
    """Comprehensive algebraic topology demonstration using HyperCat."""
    
    print("🔬 HyperCat Algebraic Topology Demo")
    print("=" * 50)
    
    # 1. Topological Spaces and Fundamental Groupoids
    print("\n1. Topological Spaces and Fundamental Groupoids")
    print("-" * 40)
    
    circle = TopologicalSpace("S¹")
    circle.points = {"base_point"}
    print(f"Space: {circle.name}")
    
    pi1_circle = circle.fundamental_groupoid()
    base = Object("*", "base_point")
    pi1_circle.add_object(base)
    
    loop = Morphism("α", base, base, {"represents": "generator of π₁(S¹) ≅ ℤ"})
    pi1_circle.add_morphism(loop)
    
    print(f"Π₁({circle.name}): {len(pi1_circle.objects)} object, {len([m for m in pi1_circle.morphisms if m.name != f'id_{m.source.name}'])} non-identity morphisms")
    
    # 2. Simplicial Sets
    print("\n2. Simplicial Sets")  
    print("-" * 20)
    
    K = SimplicalSet("K")
    K.add_simplex(0, "v₀")
    K.add_simplex(0, "v₁") 
    K.add_simplex(1, "e₀")
    K.add_simplex(1, "e₁")
    
    print(f"{K.name}: {len(K.simplices[0])} vertices, {len(K.simplices[1])} edges")
    
    K_real = K.geometric_realization()
    print(f"Geometric realization: |{K.name}| = {K_real.name}")
    
    # 3. ∞-Categories
    print("\n3. ∞-Categories")
    print("-" * 15)
    
    infinity_cat = InfinityCategory("Space_∞")
    
    X = Object("X", {"type": "topological_space"})
    Y = Object("Y", {"type": "topological_space"})  
    Z = Object("Z", {"type": "topological_space"})
    
    infinity_cat.add_object(X)
    infinity_cat.add_object(Y)
    infinity_cat.add_object(Z)
    
    f = ("f", X, Y, {"type": "continuous_map"})
    g = ("g", Y, Z, {"type": "continuous_map"})
    infinity_cat.add_n_morphism(1, f)
    infinity_cat.add_n_morphism(1, g)
    
    homotopy = ("H", f, g, {"type": "homotopy"})
    infinity_cat.add_n_morphism(2, homotopy)
    
    print(f"{infinity_cat.name}: {len(infinity_cat.objects)} objects, {len(infinity_cat.morphisms[1])} 1-morphisms, {len(infinity_cat.morphisms[2])} 2-morphisms")
    
    nerve = infinity_cat.nerve()
    print(f"Nerve: {nerve.name}")
    
    # 4. Homotopy Type Theory
    print("\n4. Homotopy Type Theory")
    print("-" * 22)
    
    unit_type = HomotopyType("𝟙", level=0)
    bool_type = HomotopyType("𝟚", level=2) 
    circle_type = HomotopyType("S¹", level=3)
    
    print(f"{unit_type.name}: contractible = {unit_type.is_contractible()}")
    print(f"{bool_type.name}: h-level {bool_type.level} (set)")
    print(f"{circle_type.name}: h-level {circle_type.level} (1-type)")
    
    path_type = HomotopyType("Path(x,y)", level=0)
    circle_type.add_path_type("x", "y", path_type)
    
    # 5. Monoidal Categories
    print("\n5. Monoidal Categories")
    print("-" * 19)
    
    vect = BraidedMonoidalCategory("Vect_k")
    
    V1 = Object("V₁", {"dimension": 2})
    V2 = Object("V₂", {"dimension": 3})
    k = Object("k", {"dimension": 1, "type": "field"})
    
    vect.add_object(V1)
    vect.add_object(V2) 
    vect.add_object(k)
    vect.set_unit_object(k)
    
    def tensor_vect(V, W):
        dim_v = V.data.get("dimension", 0)
        dim_w = W.data.get("dimension", 0)
        return Object(f"{V.name}⊗{W.name}", {"dimension": dim_v * dim_w})
    
    vect.set_tensor_product(tensor_vect)
    V1_tensor_V2 = vect.tensor_objects(V1, V2)
    
    print(f"{vect.name}: {len(vect.objects)} objects")
    print(f"Unit: {vect.unit_object.name}")
    print(f"Tensor: {V1.name} ⊗ {V2.name} = {V1_tensor_V2.name} (dim {V1_tensor_V2.data['dimension']})")
    
    braiding = Morphism(f"β_{V1.name},{V2.name}", V1_tensor_V2, tensor_vect(V2, V1), {"type": "braiding"})
    vect.set_braiding(V1, V2, braiding)
    
    # 6. Toposes
    print("\n6. Elementary Toposes")
    print("-" * 18)
    
    set_topos = Topos("Set")
    
    terminal = Object("1", {"type": "singleton_set", "elements": {"*"}})
    omega = Object("Ω", {"type": "truth_values", "elements": {"true", "false"}})
    true_morph = Morphism("true", terminal, omega, {"maps_to": "true"})
    
    set_topos.add_object(terminal)
    set_topos.add_object(omega)
    set_topos.add_morphism(true_morph)
    set_topos.set_terminal_object(terminal)
    set_topos.set_subobject_classifier(omega, true_morph)
    
    print(f"{set_topos.name}: terminal={set_topos.terminal_object.name}, subobject_classifier={set_topos.subobject_classifier.name}")
    print(f"Finite limits: {set_topos.has_finite_limits()}, exponentials: {set_topos.has_exponentials()}")
    
    # 7. Operads  
    print("\n7. Operads and Algebras")
    print("-" * 19)
    
    assoc_operad = Operad("Assoc")
    assoc_operad.add_operation(0, "unit")
    assoc_operad.add_operation(1, "identity") 
    assoc_operad.add_operation(2, "multiplication")
    assoc_operad.add_operation(3, "triple_product")
    assoc_operad.set_unit("unit")
    
    print(f"{assoc_operad.name}: operations by arity = {[(k, len(v)) for k, v in assoc_operad.operations.items()]}")
    
    monoid_obj = Object("M", {"type": "monoid", "elements": {"e", "a", "b"}})
    M_algebra = Algebra("M_alg", assoc_operad, monoid_obj)
    
    mult_map = Morphism("μ", monoid_obj, monoid_obj, {"operation": "multiplication"})
    unit_map = Morphism("η", terminal, monoid_obj, {"operation": "unit"})
    M_algebra.set_structure_map("multiplication", mult_map)
    M_algebra.set_structure_map("unit", unit_map)
    
    print(f"Algebra {M_algebra.name}: {len(M_algebra.structure_maps)} structure maps")
    
    # 8. Enriched Categories
    print("\n8. Enriched Categories")
    print("-" * 18)
    
    enriched_cat = EnrichedCategory("Set-Cat", set_topos)
    
    A = Object("A")
    B = Object("B")
    C = Object("C")
    
    for obj in [A, B, C]:
        enriched_cat.add_object(obj)
    
    hom_AB = Object("Hom(A,B)", {"morphisms": {"f₁", "f₂"}})
    hom_BC = Object("Hom(B,C)", {"morphisms": {"g₁"}})
    hom_AC = Object("Hom(A,C)", {"morphisms": {"h₁", "h₂"}})
    
    enriched_cat.set_hom_object(A, B, hom_AB)
    enriched_cat.set_hom_object(B, C, hom_BC)
    enriched_cat.set_hom_object(A, C, hom_AC)
    
    print(f"{enriched_cat.name}: {len(enriched_cat.objects)} objects, {len(enriched_cat.hom_objects)} hom-objects")
    print(f"Example: Hom(A,B) has {len(hom_AB.data['morphisms'])} morphisms")
    
    print("\n✓ Demo complete - 8 areas of algebraic topology covered")

run_algebraic_topology_demo()

In [ ]:
print("\\n" + "="*50)
print("🧪 Interactive Examples")
print("="*50)

# Simplicial triangle
print("\\nBuilding a simplicial triangle:")
triangle = SimplicalSet("Triangle")

for vertex in ["A", "B", "C"]:
    triangle.add_simplex(0, vertex)
    
for edge in ["AB", "BC", "CA"]:
    triangle.add_simplex(1, edge)
    
triangle.add_simplex(2, "ABC")

V, E, F = len(triangle.simplices[0]), len(triangle.simplices[1]), len(triangle.simplices[2])
euler_char = V - E + F

print(f"Vertices: {V}, Edges: {E}, Faces: {F}")
print(f"Euler characteristic χ = {V} - {E} + {F} = {euler_char}")

# Symmetric monoidal category of finite sets
print("\\nSymmetric monoidal category FinSet:")
finset_monoidal = SymmetricMonoidalCategory("FinSet")

empty_set = Object("∅", {"elements": set(), "size": 0})
singleton = Object("{1}", {"elements": {1}, "size": 1})  
pair = Object("{1,2}", {"elements": {1, 2}, "size": 2})

for obj in [empty_set, singleton, pair]:
    finset_monoidal.add_object(obj)

finset_monoidal.set_unit_object(empty_set)

def disjoint_union(A, B):
    elems_A = A.data["elements"]
    elems_B = B.data["elements"] 
    union_elems = {("L", x) for x in elems_A} | {("R", y) for y in elems_B}
    return Object(f"{A.name}⊔{B.name}", {"elements": union_elems, "size": len(union_elems)})

finset_monoidal.set_tensor_product(disjoint_union)

result = finset_monoidal.tensor_objects(singleton, pair)
print(f"Disjoint union: {singleton.name} ⊔ {pair.name} = {result.name}")
print(f"Size: {result.data['size']}")

braiding_morph = Morphism("swap", result, disjoint_union(pair, singleton), {"type": "iso"})
finset_monoidal.set_braiding(singleton, pair, braiding_morph)
print(f"Symmetric: {finset_monoidal.is_symmetric()}")

# Higher homotopy groups
print("\\nHigher homotopy groups of spheres:")
homotopy_data = {
    "S¹": {"π₁": "ℤ", "π₂": "0", "π₃": "0"},
    "S²": {"π₁": "0", "π₂": "ℤ", "π₃": "ℤ"}, 
    "S³": {"π₁": "0", "π₂": "0", "π₃": "ℤ"}
}

for sphere, groups in homotopy_data.items():
    print(f"{sphere}: {', '.join([f'{g}={v}' for g, v in groups.items()])}")